In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [43]:
df = pd.read_csv('product_data.csv')

In [44]:
df

,product_id,product_type,timestamp,actual_price,promotional_price,competitor1,competitor2,competitor3,competitor4,competitor5,competitor6,competitor7,competitor8,competitor9,competitor10
0,product1,electronics,1.718599e+09,1413.86,1224.30,1251.19,1269.74,1310.96,1337.50,1205.94,1266.77,1263.79,1399.98,1284.61,1304.23
1,product2,headphones,1.718573e+09,346.85,299.38,341.17,301.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,product3,beverages,1.718539e+09,2.47,2.10,2.42,2.16,2.42,2.27,2.44,2.43,2.29,2.40,2.39,2.43
3,product4,beverages,1.718508e+09,1.13,0.96,1.07,1.04,1.03,1.05,1.10,NaN,NaN,NaN,NaN,NaN
4,product5,smartphones,1.718497e+09,617.34,527.41,524.79,582.33,604.97,553.51,546.57,577.31,583.88,529.86,592.11,541.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,product996,headphones,1.674122e+09,153.78,136.97,137.72,131.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,product997,smartphones,1.674039e+09,620.15,529.28,564.97,540.74,549.48,557.77,548.65,534.21,532.65,539.18,591.07,542.85
997,product998,headphones,1.674004e+09,194.34,166.98,184.06,181.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,product999,smartphones,1.673970e+09,1096.65,941.11,1042.84,1042.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
## Label Encoding


# Example: Label encoding for product_id and competitor_id
# label_encoder = LabelEncoder()

# df['product_id'] = label_encoder.fit_transform(df['product_id'])

# # Optionally, you can also encode product_type if it's categorical
# df['product_type'] = label_encoder.fit_transform(df['product_type'])

# # Check the encoded columns
# print(df[['product_id',  'product_type']].head())

In [46]:
# ##  One-Hot Encoding

# one_hot_encoder = OneHotEncoder(sparse=False)

# # Fit-transform the data and add new columns to the DataFrame
# product_type_encoded = one_hot_encoder.fit_transform(df[['product_type']])
# product_type_encoded_df = pd.DataFrame(product_type_encoded, columns=one_hot_encoder.get_feature_names(['product_type']))

# # Concatenate with original DataFrame
# df = pd.concat([df, product_type_encoded_df], axis=1)

# # Drop original categorical column (if needed)
# df.drop(columns=['product_type'], inplace=True)

# # Check the encoded DataFrame
# print(df.head())

In [47]:
# Fill missing values with a placeholder (e.g., -1) for competitor columns
# for col in features.columns:
#     if 'competitor' in col:
#         features[col].fillna(-1, inplace=True)  # Fill missing competitor prices with -1 or any other appropriate placeholder

# features = features.fillna(features.mean())

competitor_columns = [col for col in df.columns if col.startswith('competitor')][:5]

# Fill NaN values in competitor columns with the mean of that row
# df['competitor_mean'] = df[competitor_columns].mean(axis=1)

# print(df.head())
# for col in competitor_columns:
#     df[col].fillna(df['competitor_mean'], inplace=True)

# df.drop(columns=['competitor_mean'], inplace=True)

df[competitor_columns] = df[competitor_columns].apply(lambda x: x.fillna(x.mean()), axis=1)
df['max_discount'] = np.round(df['actual_price'] - df[competitor_columns].min(axis=1), 2)

df['min_price'] = df[competitor_columns].min(axis=1)

# keep only 5 competitors
other_columns = [col for col in df.columns if not col.startswith('competitor')]
keep_columns = other_columns + competitor_columns

df_final = df[keep_columns]
# Feature engineering: Extract relevant features and target
features = df_final.drop(columns=['product_id', 'product_type', 'timestamp', 'promotional_price'])
target = df_final['promotional_price']  # Target variable: actual price

print(features)

     actual_price  max_discount  min_price  competitor1  competitor2  \
0         1413.86        207.92    1205.94      1251.19      1269.74   
1          346.85         45.14     301.71       341.17       301.71   
2            2.47          0.31       2.16         2.42         2.16   
3            1.13          0.10       1.03         1.07         1.04   
4          617.34         92.55     524.79       524.79       582.33   
..            ...           ...        ...          ...          ...   
995        153.78         22.09     131.69       137.72       131.69   
996        620.15         79.41     540.74       564.97       540.74   
997        194.34         12.86     181.48       184.06       181.48   
998       1096.65         54.36    1042.29      1042.84      1042.29   
999        144.38         20.23     124.15       138.06       124.22   

     competitor3  competitor4  competitor5  
0       1310.960     1337.500     1205.940  
1        321.440      321.440      321.440  


In [48]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

base_model = lgb.LGBMRegressor()

print(X_test)

# Fit the model
base_model.fit(X_train, y_train)

# Make predictions
y_pred = base_model.predict(X_test)

# Train the quantile regression models for lower and upper bounds
min_price_model = lgb.LGBMRegressor(objective='quantile', alpha=0.1)
max_price_model = lgb.LGBMRegressor(objective='quantile', alpha=0.9)

min_price_model.fit(X_train, y_train)
max_price_model.fit(X_train, y_train)

# Predict the quantiles
lower_bound_pred = min_price_model.predict(X_test)
upper_bound_pred = max_price_model.predict(X_test)


     actual_price  max_discount  min_price  competitor1  competitor2  \
521        219.93         22.45     197.48       197.48       197.89   
737        282.14         34.30     247.84       270.82       269.75   
740        289.15         38.25     250.90       276.68       250.90   
660          1.23          0.11       1.12         1.21         1.22   
411          1.25          0.17       1.08         1.08         1.11   
..            ...           ...        ...          ...          ...   
408        643.53         87.18     556.35       592.29       556.35   
332        584.09         80.21     503.88       503.88       526.11   
208        250.38         36.82     213.56       238.00       225.50   
613        135.54         13.25     122.29       128.37       132.29   
78         282.75         32.94     249.81       254.83       278.25   

     competitor3  competitor4  competitor5  
521      217.850      200.630      207.330  
737      264.930      247.840      262.720  


In [49]:
# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 27.302438449585868


In [50]:
# Make predictions for the entire dataset
df_predicted = df_final.copy()
df_predicted['max_predicted_price_range'] = np.round(max_price_model.predict(features),2)
df_predicted['min_predicted_price_range'] = np.round(min_price_model.predict(features),2)
df_predicted['base_predicted_price'] = np.round(base_model.predict(features),2)

# Export DataFrame to CSV with predicted promotional price
file_path_with_predictions = 'products_with_predicted_prime.csv'
df_predicted.to_csv(file_path_with_predictions, index=False)

print(f"Data with predicted prime saved to {file_path_with_predictions}")

Data with predicted prime saved to products_with_predicted_prime.csv


In [51]:
# Create the modelabs
import pickle

with open('base_model.pkl', 'wb') as f:
    pickle.dump(base_model, f)

with open('min_price_model.pkl', 'wb') as f:
    pickle.dump(min_price_model, f)

with open('max_price_model.pkl', 'wb') as f:
    pickle.dump(max_price_model, f)